In [0]:
%python
service_credential = dbutils.secrets.get(
    scope="azure-sp",
    key="client-secret"
)

adls_configs = {
    "fs.azure.account.auth.type.deproj1adls.dfs.core.windows.net": "OAuth",
    "fs.azure.account.oauth.provider.type.deproj1adls.dfs.core.windows.net": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id.deproj1adls.dfs.core.windows.net": "aca11b08-9a74-41fe-9d44-09d033504c53",
    "fs.azure.account.oauth2.client.secret.deproj1adls.dfs.core.windows.net": service_credential,
    "fs.azure.account.oauth2.client.endpoint.deproj1adls.dfs.core.windows.net": "https://login.microsoftonline.com/df6de6ef-8435-42ec-ad06-b4ca441bfdf4/oauth2/token"
}

df = spark.read.format("json").options(
    **adls_configs
).load(
    "abfss://bronze@deproj1adls.dfs.core.windows.net/rebrickable_api/Minifigs/minifigs.json"
)


In [0]:
display(df)

In [0]:
df.count()

In [0]:
from pyspark.sql.functions import explode

df2 = df.withColumn("explodedArray", explode(df.results))
display(df2)

In [0]:
df2.createOrReplaceTempView("myData")

In [0]:
%sql
SELECT 
  COUNT(*) 
FROM
  myData

In [0]:
%sql
SELECT explodedArray.last_modified_dt AS lastModifiedDatetime,
explodedArray.name AS Name,
explodedArray.num_parts AS NumberOfParts,
explodedArray.set_img_url As ImageURL,
explodedArray.set_num AS SetNumber,
explodedArray.set_url AS SetURL 
FROM myData

In [0]:
%sql
SELECT 
  to_date(explodedArray.last_modified_dt) AS lastModifiedDate,
  to_timestamp(explodedArray.last_modified_dt) AS lastModifiedDatetime,
  explodedArray.name AS Name,
  CASE 
    WHEN upper(explodedArray.name) LIKE '%TOY%' THEN 'Toy'
    WHEN upper(explodedArray.name) LIKE '%DROID%' THEN 'Droid'
    ELSE 'Other'
  END AS MinifigType,
  cast(explodedArray.num_parts AS int) AS NumberOfParts,
  explodedArray.set_img_url As ImageURL,
  explodedArray.set_num AS SetNumber,
  explodedArray.set_url AS SetURL 
FROM myData

In [0]:
%sql
SELECT
  COUNT(*)
FROM 
  myData

In [0]:
%sql
SELECT
  COUNT(DISTINCT(explodedArray.name)) AS UniqueNames
FROM 
  myData

In [0]:
%sql
--View Duplicated names
SELECT 
  to_date(explodedArray.last_modified_dt) AS lastModifiedDate,
  to_timestamp(explodedArray.last_modified_dt) AS lastModifiedDatetime,
  explodedArray.name AS Name,
  CASE 
    WHEN upper(explodedArray.name) LIKE '%TOY%' THEN 'Toy'
    WHEN upper(explodedArray.name) LIKE '%DROID%' THEN 'Droid'
    ELSE 'Other'
  END AS MinifigType,
  cast(explodedArray.num_parts AS int) AS NumberOfParts,
  explodedArray.set_img_url As ImageURL,
  explodedArray.set_num AS SetNumber,
  explodedArray.set_url AS SetURL 
FROM 
  myData
  WHERE explodedArray.name IN
  (
  SELECT 
    explodedArray.name
  FROM
    myData
  GROUP BY
    explodedArray.name
  HAVING
    COUNT(*) > 1)
   ORDER BY explodedArray.name

In [0]:
%sql
--View names without duplicates
WITH Duplicates AS (
  SELECT 
  explodedArray.last_modified_dt AS lastModifiedDateOriginal,
  to_date(explodedArray.last_modified_dt) AS lastModifiedDate,
  to_timestamp(explodedArray.last_modified_dt) AS lastModifiedDatetime,
  explodedArray.name AS Name,
  CASE 
    WHEN upper(explodedArray.name) LIKE '%TOY%' THEN 'Toy'
    WHEN upper(explodedArray.name) LIKE '%DROID%' THEN 'Droid'
    ELSE 'Other'
  END AS MinifigType,
  cast(explodedArray.num_parts AS int) AS NumberOfParts,
  coalesce(explodedArray.set_img_url, 'Not Available') As ImageURL,
  explodedArray.set_num AS SetNumber,
  explodedArray.set_url AS SetURL,
  row_number() OVER(PARTITION BY explodedArray.name ORDER BY to_timestamp(explodedArray.last_modified_dt) DESC) AS RN
FROM 
  myData
  )

SELECT * FROM Duplicates WHERE RN = 1 

In [0]:
%python
query = """
WITH Duplicates AS (
  SELECT 
    explodedArray.last_modified_dt AS lastModifiedDateOriginal,
    to_date(explodedArray.last_modified_dt) AS lastModifiedDate,
    to_timestamp(explodedArray.last_modified_dt) AS lastModifiedDatetime,
    explodedArray.name AS Name,
    CASE 
      WHEN upper(explodedArray.name) LIKE '%TOY%' THEN 'Toy'
      WHEN upper(explodedArray.name) LIKE '%DROID%' THEN 'Droid'
      ELSE 'Other'
    END AS MinifigType,
    cast(explodedArray.num_parts AS int) AS NumberOfParts,
    coalesce(explodedArray.set_img_url, 'Not Available') As ImageURL,
    explodedArray.set_num AS SetNumber,
    explodedArray.set_url AS SetURL,
    row_number() OVER(
      PARTITION BY explodedArray.name 
      ORDER BY to_timestamp(explodedArray.last_modified_dt) DESC
    ) AS RN
  FROM myData
)
SELECT * FROM Duplicates WHERE RN = 1
"""

df = spark.sql(query)
display(df)

In [0]:
df.write\
  .format("delta")\
  .options(**adls_configs)\
  .save("abfss://silver@deproj1adls.dfs.core.windows.net/rebrickable_api/Minifigs")
    